# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

Gini계수란 불순도를 측정하는 지표로서 데이터의 분산정도를 정량화한 값이다.

Gini(D) = $1 - \sum_{j=1}^{x}P_j^{2}$이다.

In [3]:
def get_gini(df, label):
    gini = 1 # gini계수를 초기값 1로 설정
    ps = df[label].value_counts() / len(df)
    # value_counts함수를 사용하여 feature의 class별 확률을 계산
    for p in ps:
        gini -= p**2
        # gini계수의 식에 따라 p**2을 빼주어 gini계수를 계산
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
from itertools import combinations

In [6]:
from itertools import combinations

def get_binary_split(df, attribute):
    result = list()
    unique = df[attribute].unique()
    for r in range(1, len(unique)): 
    # combinations함수를 활용함. nCr에서 r은 1부터 len(unique) - 1까지이다. ]
        for v in list(combinations(unique, r)): 
        # conbinations함수를 이용하여 이진분류 목록을 만듬
            result.append(list(v))
    
    return result

In [7]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [8]:
def get_attribute_gini_index(df, attribute, label):
    result = dict()
    binary = get_binary_split(df, attribute)
    n = len(df) # 가중평균을 위해 전체 sample저장
    for b in binary:
        tmp_df1 = df[df[attribute].isin(b)] # binary split 중 b를 포함한 경우
        tmp_df2 = df[~df[attribute].isin(b)] # binary split 중 b를 포함하지 않은 경우
        gini_index = len(tmp_df1) / n * get_gini(tmp_df1, label) + len(tmp_df2) / n * get_gini(tmp_df2, label)
        # 위에서 정의한 get_gini함수를 사용하여 gini계수를 계산하고 가중평균하여 gini index를 계산
        result['_'.join(b)] = gini_index
        
    return result

In [9]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [10]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [11]:
##문제1 답안
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [12]:
##문제2 답안

def decide_feature_class(df, features, label):
    min_feature = "" # gini_index가 최소일때의 feature
    min_class = "" # gini_index가 최소일때의 class
    min_gini_index = 9999999 # gini_index의 최소값
    for c in features:
        tmp = min(get_attribute_gini_index(df, c, label).items())
        # 위에서 정의한 get_attribute_gini_index함수를 사용해 feature에서의 최소 gini_index를 찾는다.
        if tmp[1] <= min_gini_index:
            min_features = c
            min_class = tmp[0]
            min_gini_index = tmp[1]
        # 현재의 min_gini_index보다 더 작은 gini_index를 찾으면 update해준다.
    return min_features, min_class, min_gini_index

label = "class_buys_computer"
features = pd_data.columns[pd_data.columns != label]
min_feature, min_class, min_gini_index = decide_feature_class(pd_data, features, label)
# label인 "class_buys_computer"를 제외한 나머지 feature에 대해 탐색
print("feature : %s, class : %s, gini_index : %s"%(min_feature, min_class, min_gini_index))

feature : age, class : middle_aged, gini_index : 0.35714285714285715


In [13]:
##문제3 답안

left_child = pd_data[pd_data[min_feature] == min_class]
# 문제 2의 min_class가 포함된 데이터프레임
right_child = pd_data[pd_data[min_feature] != min_class]
# 문제 2의 min_class가 포함되지 않은 데이터프레임

left_min_feature, left_min_class, left_min_gini_index = decide_feature_class(left_child, features[features != min_feature], label)
right_min_feature, right_min_class, right_min_gini_index = decide_feature_class(right_child, features[features != min_feature], label)
# 부모노드에서 사용한 feature(min_feature)를 제외하고 나머지 feature들 중 
# decide_feature_class를 사용하여 그 다음 분기할 feature와 class를 탐색

In [14]:
print("left_child_feature : %s,\nleft_child_class : %s,\nleft_child_gini_index : %s"%(left_min_feature, left_min_class, left_min_gini_index))

left_child_feature : credit_rating,
left_child_class : excellent,
left_child_gini_index : 0.0


In [15]:
print("right_child_feature : %s,\nright_child_class : %s,\nright_child_gini_index : %s"%(right_min_feature, right_min_class, right_min_gini_index))

right_child_feature : student,
right_child_class : no,
right_child_gini_index : 0.31999999999999984
